## 讀取模型

這邊讀取 Llama-2 7B Chat 模型
* 因為原本的 Llama-2 需要權限並設定 SSH 公鑰才能直接下載，因此這邊使用 TheBloke 大神上傳的 FP16 版本做示範
    * [TheBloke](https://huggingface.co/TheBloke) 是 HuggingFace 社群上非常活躍的用戶，經常幫各個模型做 Quantization 並重新上傳

#### 使用 .from_pretrained 讀取： 
* *TheBloke 大神提供的 [Llama-2 7B](https://huggingface.co/TheBloke/Llama-2-7b-chat-fp16) Chat 為例：
    * 使用**量化 (Quantization)** 技術來減少 GPU 記憶體的消耗
    * 透過 `BitsAndBytesConfig` 可以輕鬆地將模型量化 8 到 4 位元。
    * 將 `device_map` 設定為 `"auto"`，以確保模型權重都放在 GPU 裡面。
    * 如果 CPU 記憶體也不是很夠的話，那就需要將 `low_cpu_mem_usage` 設定為 `True` 才能順利讀取模型。
* 以上步驟實際上會從 [Hugging Face Hub](https://huggingface.co/models) (HF Hub)下載模型的權重，預設會放在 `~/.cache/huggingface/` 裡面，也可以透過`cache_dir` 參數指定這些檔案要存放在哪裡：

In [1]:
from transformers import BitsAndBytesConfig
from transformers import LlamaForCausalLM

model_path = "TheBloke/Llama-2-7b-chat-fp16"

# 需要 CUDA 與 GPU
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,    # 約需 8 GiB GPU 記憶體
    # load_in_4bit=True,  # 約需 6 GiB GPU 記憶體
)

# 需要 CUDA 與 GPU
model = LlamaForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    low_cpu_mem_usage=True,
    quantization_config=bnb_config,
)

# # 無 GPU 指定用 CPU
# model = LlamaForCausalLM.from_pretrained(
#     model_path,

#     # device_map="auto", 在無 GPU 環境中沒意義。
#     device_map={"": "cpu"},

#     # 這是 Hugging Face 為大型模型設計的「省 RAM 載入技巧」，需要 PyTorch 2.0+ 和 GPU。
#     # 在無 GPU 時常會導致奇怪的錯誤或佔用更多 CPU 資源。
#     # low_cpu_mem_usage=True,

#     # 完全不能用，因為 bnb_config 是屬於 bitsandbytes 的，該套件需要 GPU + CUDA。
#     # quantization_config=bnb_config,
# )

# # 指定下載的模型權重檔案會放在哪
# model = LlamaForCausalLM.from_pretrained(
#     model_path,
#     cache_dir="./cache_models"
# )

/home/ai-x/miniconda3/envs/ann_py311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.27s/it]
/home/ai-x/miniconda3/envs/ann_py311/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/ai-x/miniconda3/envs/ann_py311/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. 

查看本地模型的 commit id

In [6]:
%%bash
ls ~/.cache/huggingface/hub/models--TheBloke--Llama-2-7b-chat-fp16/snapshots/

4594bf11fd5c66a9021d20f5ae5f3a246ce388a1


查看目前的 generation_config
* 如果出現 `do_sample` 的警告訊息，那是因為預設的 `do_sample` 為 `False`，與模型內建的 `generation_config.json` 有衝突設定。

In [2]:
from transformers import GenerationConfig

print(model.generation_config)

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.9,
  "top_p": 0.6
}



`generation_config.json`
* 位置通常在：
    * `~/.cache/huggingface/hub/models--<model-name>/snapshots/<commit-id>/generation_config.json`
* 可以使用 `find ~/.cache/huggingface/hub -name generation_config.json` 尋找，並進行修改。
    * 加上 `"do_sample": true,`，或是刪除 `temperature` 與 `top_p` 等參數。


In [9]:
%%bash
find ~/.cache/huggingface/hub -name generation_config.json

/home/ai-x/.cache/huggingface/hub/models--TheBloke--Llama-2-7b-chat-fp16/snapshots/4594bf11fd5c66a9021d20f5ae5f3a246ce388a1/generation_config.json


查看安裝位置

In [5]:
%%bash

echo "~/.cache/huggingface:"
ls -al ~/.cache/huggingface

echo "~/.cache/huggingface/hub:"
ls -al ~/.cache/huggingface/hub

~/.cache/huggingface:
total 24
drwxrwxr-x  4 ai-x docker 4096  四   7 17:14 .
drwx------ 20 ai-x ai-x   4096  五  15 16:58 ..
drwxrwxr-x  2 ai-x ai-x   4096  四   7 17:14 accelerate
drwxrwxr-x  9 ai-x docker 4096  五  16 09:13 hub
-rw-rw-r--  1 ai-x ai-x     68  四   9 09:58 stored_tokens
-rw-rw-r--  1 ai-x ai-x     37  四   9 09:58 token
~/.cache/huggingface/hub:
total 40
drwxrwxr-x 9 ai-x docker 4096  五  16 09:13 .
drwxrwxr-x 4 ai-x docker 4096  四   7 17:14 ..
drwxrwxr-x 9 ai-x docker 4096  五  16 09:13 .locks
drwxrwxr-x 6 ai-x docker 4096  三  31 18:13 models--BAAI--bge-reranker-v2-m3
drwxrwxr-x 6 ai-x ai-x   4096  五  15 11:59 models--intfloat--multilingual-e5-base
drwxrwxr-x 6 ai-x docker 4096  三  31 18:09 models--sentence-transformers--all-MiniLM-L6-v2
drwxrwxr-x 6 ai-x ai-x   4096  四  30 11:39 models--sentence-transformers--paraphrase-multilingual-MiniLM-L12-v2
drwxrwxr-x 6 ai-x ai-x   4096  五  16 09:16 models--TheBloke--Llama-2-7b-chat-fp16
drwxrwxr-x 3 ai-x ai-x   4096  四   1 12:07 mod

#### 使用 Git 取得 [Hugging Face Hub](https://huggingface.co/models) (HF Hub) 中的模型的權重

1. 安裝 Git LFS 模組

In [ ]:
%%bash
sudo apt install -y git git-lfs

2. 啟動 Git LFS 功能

In [ ]:
%%bash
git lfs install

3. 下載模型

In [ ]:
%%bash
# 直接複製
git clone https://huggingface.co/TheBloke/Llama-2-7b-chat-fp16

# # 如果想看下載的網速圖紙紙條
# # 1. 跳過所有 LFS 檔案的下載
# GIT_LFS_SKIP_SMUDGE=1 git clone https://huggingface.co/TheBloke/Llama-2-7b-chat-fp16
# cd Llama-2-7b-chat-fp16
# # 2. 透過 Git LFS 進行 Fetch
# git lfs fetch
# git lfs checkout

4. 讀取模型

In [ ]:
from transformers import LlamaForCausalLM

model = LlamaForCausalLM.from_pretrained("TheBloke/Llama-2-7b-chat-fp16")


In [1]:
from transformers import LlamaForCausalLM

# 直接指定資料夾路徑名稱
model = LlamaForCausalLM.from_pretrained(
    "TheBloke/Llama-2-7b-chat-fp16",
    cache_dir="./cache_models"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

: 

: 

### 有存取限制的模型
* 先去 [HF Hub](https://huggingface.co/) 註冊帳號
* 進入該模型介紹頁面，找到 "This repository is gated" 區塊，填寫存取表單，並等待核准通知。
* 前往 HF 帳號設定的 Access Tokens 頁面，並建立一個新的 Token。
    * 如果只要下載模型的話，權限只要設定 Read 就好。
    * 如果你想讓 Git 也記得這個 token，避免每次 push 都輸入密碼，可以執行：
        ```
        git config --global credential.helper store
        ```
        這會讓 Git 把憑證儲存在本地 .git-credentials 中（純文字，請小心安全性）。
        * 詳細說明：https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage
* 使用建立好的 Access Token 登入後下載模型。


(方式一) 設定 SSH 金鑰，然後透過 SSH 下載：

In [ ]:
%%bash
git clone git@hf.co:meta-llama/Meta-Llama-3-8B

(方式二) 透過 `huggingface-cli` 登入：

In [ ]:
%%bash

huggingface-cli login
# 貼上 建立好的 Access Token

# 預設會放在 `~/.cache/huggingface/` 裡面，也可以透過 `--local-dir` 參數指定這些檔案要存放在哪裡
huggingface-cli download google/gemma-2b-it \
    --local-dir Models/Meta-Llama-3-8B